In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from pathlib import Path
import sklearn as skl
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

import pickle

In [2]:
stock_data = Path('StockAIClean.csv')
stock_df = pd.read_csv(stock_data)
stock_df.head()

,No.,Ticker,Company,Sector,Mkt Cap Cat,For P/E Cat,Dividend,EPS growth this year (%) Cat,EPS growth next year (%) Cat,EPS growth past 5 years (%) Cat,EPS growth next 5 years (%) Cat,Sales growth past 5 years (%) Cat,Sales Cat,Float Short (%) Cat,Profit Margin (%) Cat,Performance (Year) (%) Cat,Employees Cat,Analyst Rec Cat,Volume Cat,Var % Cat
0,1,AAPL,Apple Inc.,Technology,Mg Cap,Med,Yes,Yes,Yes,Yes,Yes,Yes,Hi,Lo,Pos,Up,Hi,Buy,Hi,Neg
1,2,ACN,Accenture plc,Technology,Lg Cap,Med,Yes,Yes,Yes,Yes,Yes,Yes,Hi,Lo,Pos,Up,Hi,Buy,Med,Pos
2,3,ADBE,Adobe Inc.,Technology,Lg Cap,Hi,No,Yes,Yes,Yes,Yes,Yes,Med,Lo,Pos,Up,Med,Buy,Med,Neg
3,4,ADSK,"Autodesk, Inc.",Technology,Lg Cap,Med,No,Yes,Yes,Yes,Yes,Yes,Med,Lo,Pos,Down,Med,Buy,Lo,Pos
4,5,AI,"C3.ai, Inc.",Technology,Mid Cap,Hi,No,No,Yes,No,Yes,Yes,Lo,Hi,Neg,Up,Sm,Hold,Hi,Neg


In [3]:
len(stock_df)

72

In [4]:
stock_df1 = stock_df.drop(columns=['No.', 'Ticker', 'Company', 'Sector'])
stock_df1.head()

,Mkt Cap Cat,For P/E Cat,Dividend,EPS growth this year (%) Cat,EPS growth next year (%) Cat,EPS growth past 5 years (%) Cat,EPS growth next 5 years (%) Cat,Sales growth past 5 years (%) Cat,Sales Cat,Float Short (%) Cat,Profit Margin (%) Cat,Performance (Year) (%) Cat,Employees Cat,Analyst Rec Cat,Volume Cat,Var % Cat
0,Mg Cap,Med,Yes,Yes,Yes,Yes,Yes,Yes,Hi,Lo,Pos,Up,Hi,Buy,Hi,Neg
1,Lg Cap,Med,Yes,Yes,Yes,Yes,Yes,Yes,Hi,Lo,Pos,Up,Hi,Buy,Med,Pos
2,Lg Cap,Hi,No,Yes,Yes,Yes,Yes,Yes,Med,Lo,Pos,Up,Med,Buy,Med,Neg
3,Lg Cap,Med,No,Yes,Yes,Yes,Yes,Yes,Med,Lo,Pos,Down,Med,Buy,Lo,Pos
4,Mid Cap,Hi,No,No,Yes,No,Yes,Yes,Lo,Hi,Neg,Up,Sm,Hold,Hi,Neg


In [5]:
stock_df2 = pd.get_dummies(stock_df1, columns=["Mkt Cap Cat", "For P/E Cat", "Dividend", "EPS growth this year (%) Cat", "EPS growth next year (%) Cat", "EPS growth past 5 years (%) Cat", "EPS growth next 5 years (%) Cat", "Sales growth past 5 years (%) Cat", "Sales Cat", "Float Short (%) Cat", "Profit Margin (%) Cat", "Performance (Year) (%) Cat", "Employees Cat", "Volume Cat", "Var % Cat"])
stock_df2

,Analyst Rec Cat,Mkt Cap Cat_Lg Cap,Mkt Cap Cat_Mg Cap,Mkt Cap Cat_Mic Cap,Mkt Cap Cat_Mid Cap,Mkt Cap Cat_Sm Cap,For P/E Cat_Hi,For P/E Cat_Low,For P/E Cat_Med,Dividend_No,...,Performance (Year) (%) Cat_Down,Performance (Year) (%) Cat_Up,Employees Cat_Hi,Employees Cat_Med,Employees Cat_Sm,Volume Cat_Hi,Volume Cat_Lo,Volume Cat_Med,Var % Cat_Neg,Var % Cat_Pos
0,Buy,0,1,0,0,0,0,0,1,0,...,0,1,1,0,0,1,0,0,1,0
1,Buy,1,0,0,0,0,0,0,1,0,...,0,1,1,0,0,0,0,1,0,1
2,Buy,1,0,0,0,0,1,0,0,1,...,0,1,0,1,0,0,0,1,1,0
3,Buy,1,0,0,0,0,0,0,1,1,...,1,0,0,1,0,0,1,0,0,1
4,Hold,0,0,0,1,0,1,0,0,1,...,0,1,0,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Buy,1,0,0,0,0,1,0,0,1,...,0,1,0,1,0,1,0,0,0,1
68,Hold,0,0,0,1,0,1,0,0,1,...,0,1,0,0,1,1,0,0,1,0
69,Hold,0,0,1,0,0,0,1,0,1,...,1,0,0,0,1,0,1,0,0,1
70,Buy,1,0,0,0,0,1,0,0,0,...,0,1,0,1,0,0,0,1,0,1


In [6]:
X = stock_df2.drop(columns=['Analyst Rec Cat'])
y = stock_df2['Analyst Rec Cat']

In [7]:
y

0      Buy
1      Buy
2      Buy
3      Buy
4     Hold
      ... 
67     Buy
68    Hold
69    Hold
70     Buy
71     Buy
Name: Analyst Rec Cat, Length: 72, dtype: object

In [8]:
X.columns

Index(['Mkt Cap Cat_Lg Cap', 'Mkt Cap Cat_Mg Cap', 'Mkt Cap Cat_Mic Cap',
       'Mkt Cap Cat_Mid Cap', 'Mkt Cap Cat_Sm Cap', 'For P/E Cat_Hi',
       'For P/E Cat_Low', 'For P/E Cat_Med', 'Dividend_No', 'Dividend_Yes',
       'EPS growth this year (%) Cat_No', 'EPS growth this year (%) Cat_Yes',
       'EPS growth next year (%) Cat_No', 'EPS growth next year (%) Cat_Yes',
       'EPS growth past 5 years (%) Cat_No',
       'EPS growth past 5 years (%) Cat_Yes',
       'EPS growth next 5 years (%) Cat_No',
       'EPS growth next 5 years (%) Cat_Yes',
       'Sales growth past 5 years (%) Cat_No',
       'Sales growth past 5 years (%) Cat_Yes', 'Sales Cat_Hi', 'Sales Cat_Lo',
       'Sales Cat_Med', 'Float Short (%) Cat_Hi', 'Float Short (%) Cat_Lo',
       'Float Short (%) Cat_Med', 'Profit Margin (%) Cat_Neg',
       'Profit Margin (%) Cat_Pos', 'Performance (Year) (%) Cat_Down',
       'Performance (Year) (%) Cat_Up', 'Employees Cat_Hi',
       'Employees Cat_Med', 'Employees Cat_Sm

In [9]:
stock_df2.isnull()

,Analyst Rec Cat,Mkt Cap Cat_Lg Cap,Mkt Cap Cat_Mg Cap,Mkt Cap Cat_Mic Cap,Mkt Cap Cat_Mid Cap,Mkt Cap Cat_Sm Cap,For P/E Cat_Hi,For P/E Cat_Low,For P/E Cat_Med,Dividend_No,...,Performance (Year) (%) Cat_Down,Performance (Year) (%) Cat_Up,Employees Cat_Hi,Employees Cat_Med,Employees Cat_Sm,Volume Cat_Hi,Volume Cat_Lo,Volume Cat_Med,Var % Cat_Neg,Var % Cat_Pos
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
68,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
69,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
70,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [10]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1, stratify=y)

In [11]:
# create a scaler instance
X_scaler = skl.preprocessing.StandardScaler()

In [12]:
# fit/train the scaler
X_scaler.fit(X_train)

StandardScaler()

In [13]:
# scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# create a logistic regression model
#model = LogisticRegression(solver='lbfgs', random_state=1)
from sklearn.linear_model import LogisticRegressionCV
classifier = LogisticRegressionCV(cv=5, penalty='l1', solver='saga', 
            max_iter=10000, random_state=1)
classifier

LogisticRegressionCV(cv=5, max_iter=10000, penalty='l1', random_state=1,
                     solver='saga')

In [15]:
classifier.fit(X_train, y_train)

LogisticRegressionCV(cv=5, max_iter=10000, penalty='l1', random_state=1,
                     solver='saga')

In [16]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.8703703703703703
Testing Data Score: 0.8333333333333334


In [17]:
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,Buy,Buy
1,Buy,Hold
2,Buy,Buy
3,Buy,Buy
4,Buy,Buy
5,Buy,Buy
6,Buy,Buy
7,Buy,Buy
8,Buy,Buy
9,Buy,Hold


In [18]:
# Display the accuracy score for the test dataset.
score = balanced_accuracy_score(y_test, predictions)
print('Accuracy score: ', score)

Accuracy score:  0.5


In [19]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,15,0
Actual 1,3,0


In [20]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         Buy       0.83      1.00      0.91        15
        Hold       0.00      0.00      0.00         3

    accuracy                           0.83        18
   macro avg       0.42      0.50      0.45        18
weighted avg       0.69      0.83      0.76        18



c:\Users\soobi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\soobi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\soobi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resample, y_resample = ros.fit_resample(X_train, y_train)

Counter(y_resample)
y_resample.value_counts()


Buy     47
Hold    47
Name: Analyst Rec Cat, dtype: int64

In [22]:
classifier1 = LogisticRegressionCV(cv=5, penalty='l1', solver='saga', 
            max_iter=10000, random_state=1)
classifier1

LogisticRegressionCV(cv=5, max_iter=10000, penalty='l1', random_state=1,
                     solver='saga')

In [23]:
# Fit the model using the resampled training data
classifier1.fit(X_resample, y_resample)

LogisticRegressionCV(cv=5, max_iter=10000, penalty='l1', random_state=1,
                     solver='saga')

In [24]:
# Make a prediction using the testing data
predictions1= classifier1.predict(X_test)
pd.DataFrame({"Prediction": predictions1, "Actual": y_test})

,Prediction,Actual
18,Buy,Buy
61,Hold,Hold
8,Buy,Buy
1,Buy,Buy
10,Buy,Buy
12,Buy,Buy
16,Buy,Buy
49,Buy,Buy
7,Buy,Buy
69,Buy,Hold


In [25]:
# Print the balanced_accuracy score of the model 
score1 = balanced_accuracy_score(y_test, predictions1)
print('New accuracy score: ', score1)


New accuracy score:  0.8333333333333333


In [26]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions1)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,15,0
Actual 1,1,2


In [27]:
print(classification_report(y_test, predictions1))

              precision    recall  f1-score   support

         Buy       0.94      1.00      0.97        15
        Hold       1.00      0.67      0.80         3

    accuracy                           0.94        18
   macro avg       0.97      0.83      0.88        18
weighted avg       0.95      0.94      0.94        18



In [28]:
import pickle

with open("classifier.pkl", "wb") as f:
    pickle.dump(classifier,f)

with open("classifier1.pkl", "wb") as f:
    pickle.dump(classifier1,f)

In [29]:
# # save the model to curent directory
# filename = 'classifier1.pkl'
# pickle.dump(classifier1, open(filename, 'wb'))

In [30]:
# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, y_test)
# loaded_model